# 종목별 네이버 기사 크롤링

# 삼성증권 으로 테스트

In [4]:
import requests
from bs4 import BeautifulSoup
import math
import re
import pandas as pd
import os
import time

In [7]:
Url = 'https://search.naver.com/search.naver?'
# 'where=news&sm=tab_pge&query=%EC%82%BC%EC%84%B1%EC%A6%9D%EA%B6%8C&ds=2020.01.01&de=2021.11.30&mynews=1&news_office_checked=1032&nso=so:r,p:from20200101to20211130,a:all&start=31

In [10]:
key_word = "삼성증권"

In [2]:
params = {
    "where" : 'news',
    "sm": 'tab_pge',
    "query": key_word,  # ★인코딩하지말고 넣어라★
    "sort": '0',
    "photo": '0',
    "field": '0',
    "pd": '3',
    "ds": '2021.01.01',
    "de" : '2021.11.30',
    "cluster_rank": '10',
    "mynews" : '1',
    "office_type": '0',
    "office_section_code": '0',
    "news_office_checked" : '1032',
    "nso" : r'so:r,p:from20210101to20211130,a:all',
    "start" : '1000'
}

# 경향신문, 국민일보, 동아일보, 한겨례, 매일일보, 조선일보
news_code = ['1032', '1005', '1020', '1028', '2385', '1023']

NameError: name 'key_word' is not defined

In [9]:
urls = []

for code in news_code:
  params['news_office_checked'] = code
  raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

  dom = BeautifulSoup(raw.text, "html.parser")

  a_tags = dom.select('#main_pack > div.api_sc_page_wrap > div > div > a')
  last_page = int(a_tags[-1].text.strip())

  for start in range(1, last_page * 10, 10):
    print(start)
    params['start'] = start
    raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

    dom = BeautifulSoup(raw.text, "html.parser")

    lists = dom.select("#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li")

    for l in lists:
      urls.append(l.select("div.news_wrap > div.news_area > div.news_info > div.info_group > a")[-1]['href'])
    
    print(code, '의', start, '번째 완료!')
    time.sleep(2)

1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31


KeyboardInterrupt: 

In [ ]:
# 1, 11, 21, 31, 41, 51, 61, 71, 81

In [ ]:
len(urls)

698

In [ ]:
news_urls = [
             url
             for url in urls
             if 'https://news.naver.com/' in url
]
len(news_urls)

556

In [ ]:
news = []
i = 1
for url in news_urls:
  
  raw = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

  dom = BeautifulSoup(raw.text, "html.parser")
  
  title = dom.select_one('#articleTitle').text.strip()
  date = dom.select_one('#main_content > div.article_header > div.article_info > div > span.t11').text.strip()[:10]

  # <script> <style> 제거 (전처리)
  for s in dom(['script', 'style', 'img']):
      s.decompose()
  
  # 뉴스본문 리턴
  news_content = dom.find('div', attrs = {'id': 'articleBodyContents'})
  # 뉴스본문에 대한 전처리
  # 각 line 별로 strip(), 태그 제거
  lines = [
      line.strip()
      for line in news_content.get_text().splitlines()
  ]
  news_content = ''.join(lines).replace('[경향신문]','')
  # content = "".join([
  #     line + "."
  #     for line in news_content.split('.')
  #     if "삼성증권" in line
  # ])

  dic = {
      'title' : title,
      'date' : date,
      'content' : news_content
  }
  news.append(dic)

  print(str(i) + '번째 기사 완료!!')
  i += 1
  time.sleep(2)

In [ ]:
df = pd.DataFrame(news)

base_path = r'C:\Users\young\Desktop\StockHelper\data'  # 경로 바꿔서 사용해주세요!!
file_path = os.path.join(base_path, f"{key_word}_기사_크롤링_{params['ds']}_{params['de']}.csv")
# file_path = os.path.join(base_path, '삼성증권_기사_크롤링_테스트.csv')

df.to_csv(base_path, encoding='utf-8-sig')

In [ ]:
print(urls[226])

https://sports.news.naver.com/news.nhn?oid=020&aid=0003291416


In [17]:
import requests
from bs4 import BeautifulSoup
import math
import re
import pandas as pd
import os
import time

def crawling_article(key_word):
  Url = 'https://search.naver.com/search.naver?'
  ds = ['2020.01.01', '2021.01.01']
  de = ['2020.12.31', '2021.12.06']  # 두번째 날은 어제로 설정
  
  params = {
      "where" : 'news',
      "sm": 'tab_pge',
      "query": key_word,  # ★인코딩하지말고 넣어라★
      "sort": '0',
      "photo": '0',
      "field": '0',
      "pd": '3',
      "ds": '2020.01.01',
      "de" : '2020.12.31',
      "cluster_rank": '10',
      "mynews" : '1',
      "office_type": '0',
      "office_section_code": '0',
      "news_office_checked" : '1032',
      "nso" : r'so:r,p:from20210101to20211130,a:all',
      "start" : '5000'
  }

  # 경향신문, 국민일보, 동아일보, 한겨례, 매일일보, 조선일보
  news_code = ['1032', '1005', '1020', '1028', '2385', '1023']

  # 크롤링 할 url 저장
  urls = []

  for i in range(2):
    # 날짜 설정
    params['ds'] = ds[i]
    params['de'] = de[i]

    for code in news_code:
      params['news_office_checked'] = code
      raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

      dom = BeautifulSoup(raw.text, "html.parser")

      a_tags = dom.select('#main_pack > div.api_sc_page_wrap > div > div > a')
      last_page = int(a_tags[-1].text.strip())

      for start in range(1, last_page * 10, 10):
        print(start)
        params['start'] = start
        raw = requests.get(Url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)

        dom = BeautifulSoup(raw.text, "html.parser")

        lists = dom.select("#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li")

        for l in lists:
          urls.append(l.select("div.news_wrap > div.news_area > div.news_info > div.info_group > a")[-1]['href'])
        
        print(code, '의', start, '번째 완료!')
        time.sleep(1)

  # 기사 총 몇 개 클롤링 하는지 보여주기
  news_urls = [
             url
             for url in urls
             if 'https://news.naver.com/' in url
  ]
  print(len(news_urls))

  # 크롤링한 날짜, 제목, 본문 저장
  news = []

  i = 1 # 몇 번째 기사를 크롤링하는지 체크
  
  for url in news_urls:
    print(url)
    raw = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

    dom = BeautifulSoup(raw.text, "html.parser")
    
    title = dom.select_one('#articleTitle').text.strip()
    date = dom.select_one('#main_content > div.article_header > div.article_info > div > span.t11').text.strip()[:10]

    # <script> <style> 제거 (전처리)
    for s in dom(['script', 'style', 'img']):
        s.decompose()
    
    # 뉴스본문 리턴
    news_content = dom.find('div', attrs = {'id': 'articleBodyContents'})
    # 뉴스본문에 대한 전처리
    # 각 line 별로 strip(), 태그 제거
    lines = [
        line.strip()
        for line in news_content.get_text().splitlines()
    ]
    news_content = ''.join(lines).replace('[경향신문]','')

    dic = {
        'title' : title,
        'date' : date,
        'content' : news_content
    }
    news.append(dic)

    print(str(i) + '번째 기사 완료!!')
    i += 1
    time.sleep(1)

  df = pd.DataFrame(news)

  base_path = r'../../dataset'
  file_path = os.path.join(base_path, f"{key_word}_기사_크롤링_2020.01.01_{params['de']}.csv")

  df.to_csv(file_path, encoding='utf-8-sig')

In [3]:
# 전체 주식 정보 가져오기
import FinanceDataReader as fdr

df = fdr.StockListing("KRX")
df

# 다음 시가총액 TOP 100 읽어오기
import requests
from bs4 import BeautifulSoup

# 다음 시가총액 TOP 100
url = "https://finance.daum.net/api/trend/market_capitalization?page={}&perPage=30&fieldName=marketCap&order=desc&market=KOSPI&pagination=true"

headers={
    "User-Agent" : "Mozilla/5.0",
    "referer" : "https://finance.daum.net/domestic/market_cap?market=KOSPI"
}

stock_name = []
for i in range(1, 5):
    response = requests.get(url.format(i), headers=headers).json()  # json
    data = response['data']

    if i < 4:
        for j in range(30):
            stock_name.append(data[j]['name'])
    else:
        for j in range(10):
            stock_name.append(data[j]['name'])
            
# stock_name.extend(['포스코', '케이티', '케이티앤지', '에스케이바이오팜', '금호석유화학','삼성화재해상보험', '쌍용씨앤이', '아모레퍼시픽그룹', '한국전력공사', '현대자동차'])
print(stock_name)

stock_name_df = pd.DataFrame(stock_name)

# 시가총액 TOP 100 중 상장일이 2020-01-01 이전인 주식 정보 가져오기
stock_df = df[df['Name'].isin(stock_name)]
stock_df = stock_df[stock_df['ListingDate']<='2020-01-01']
stock_df['Name']

['삼성전자', 'SK하이닉스', 'NAVER', '삼성전자우', '삼성바이오로직스', '카카오', 'LG화학', '삼성SDI', '현대차', '기아', '카카오뱅크', '셀트리온', '카카오페이', 'POSCO', 'KB금융', '현대모비스', '크래프톤', '삼성물산', 'LG전자', '신한지주', 'SK이노베이션', 'SK', 'LG생활건강', 'SK바이오사이언스', '엔씨소프트', '한국전력', '삼성전기', '삼성생명', 'HMM', '하이브', '하나금융지주', 'LG', 'SK텔레콤', '삼성에스디에스', 'SK아이이테크놀로지', 'KT&G', '포스코케미칼', '두산중공업', '넷마블', '아모레퍼시픽', 'S-Oil', '대한항공', '삼성화재', '현대중공업', '우리금융지주', 'SK스퀘어', '고려아연', '기업은행', 'KT', 'LG디스플레이', '롯데케미칼', 'SK바이오팜', '한온시스템', 'SKC', '한화솔루션', '한국조선해양', 'LG이노텍', 'F&F', 'LG유플러스', '현대글로비스', '미래에셋증권', '코웨이', '현대제철', '현대건설', '맥쿼리인프라', '일진머티리얼즈', 'CJ제일제당', '금호석유', '에스디바이오센서', '강원랜드', 'KODEX 200', '한국타이어앤테크놀로지', '삼성중공업', '메리츠금융지주', '한국금융지주', '현대중공업지주', '삼성엔지니어링', '삼성증권', '유한양행', '두산밥캣', '이마트', '한진칼', '오리온', 'DB손해보험', '메리츠화재', '쌍용C&E', '삼성카드', 'NH투자증권', '아모레G', 'GS', '현대차2우B', '한미사이언스', '메리츠증권', '한솔케미칼', 'GS건설', '한국가스공사', '한전기술', 'GS리테일', '한미약품', '롯데지주']


63       CJ제일제당
77       DB손해보험
114          GS
116        GS건설
118       GS리테일
         ...   
7000     현대글로비스
7005      현대모비스
7027       현대제철
7029    현대중공업지주
7030        현대차
Name: Name, Length: 86, dtype: object

In [5]:
import random

stock_list = list(stock_df['Name'])
stock_list.pop(stock_list.index('삼성증권'))
random.shuffle(stock_list)  # 순서 바꿔주기
print(stock_list)
len(stock_list)

['메리츠화재', 'SK', '한미사이언스', '한진칼', 'LG전자', '일진머티리얼즈', '현대중공업지주', '넷마블', '오리온', '신한지주', '고려아연', 'LG디스플레이', '삼성에스디에스', 'SK텔레콤', 'LG생활건강', 'LG유플러스', '삼성엔지니어링', '한솔케미칼', '맥쿼리인프라', '한온시스템', '미래에셋증권', 'SK이노베이션', 'POSCO', '롯데케미칼', '두산밥캣', 'LG', '삼성전기', '이마트', '삼성바이오로직스', 'LG화학', 'GS', '삼성전자', '삼성중공업', '한국금융지주', '아모레G', 'CJ제일제당', '대한항공', '하나금융지주', '금호석유', 'GS건설', '삼성화재', '셀트리온', '한화솔루션', '유한양행', 'HMM', '한국가스공사', '현대글로비스', 'KT&G', '우리금융지주', 'DB손해보험', '삼성물산', '삼성SDI', 'KB금융', '메리츠증권', '현대차', '한국타이어앤테크놀로지', '강원랜드', '쌍용C&E', '삼성생명', '포스코케미칼', 'GS리테일', '삼성카드', '현대건설', '한전기술', '기아', '카카오', '기업은행', '엔씨소프트', 'NAVER', 'LG이노텍', '한국전력', '아모레퍼시픽', 'SKC', '한국조선해양', '두산중공업', '한미약품', 'S-Oil', 'NH투자증권', '메리츠금융지주', '코웨이', 'KT', 'SK하이닉스', '롯데지주', '현대모비스', '현대제철']


85

In [ ]:
crawling_article('LG화학')

In [ ]:
crawling_article('메리츠화재')

In [ ]:
crawling_article('삼성엔지니어링')

In [14]:
crawling_article('한국가스공사')

1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31
1032 의 31 번째 완료!
41
1032 의 41 번째 완료!
51
1032 의 51 번째 완료!
61
1032 의 61 번째 완료!
71
1032 의 71 번째 완료!
1
1005 의 1 번째 완료!
11
1005 의 11 번째 완료!
21
1005 의 21 번째 완료!
31
1005 의 31 번째 완료!
41
1005 의 41 번째 완료!
51
1005 의 51 번째 완료!
61
1005 의 61 번째 완료!
71
1005 의 71 번째 완료!
81
1005 의 81 번째 완료!
1
1020 의 1 번째 완료!
11
1020 의 11 번째 완료!
21
1020 의 21 번째 완료!
31
1020 의 31 번째 완료!
41
1020 의 41 번째 완료!
51
1020 의 51 번째 완료!
61
1020 의 61 번째 완료!
1
1028 의 1 번째 완료!
11
1028 의 11 번째 완료!
21
1028 의 21 번째 완료!
31
1028 의 31 번째 완료!
41
1028 의 41 번째 완료!
51
1028 의 51 번째 완료!
1
2385 의 1 번째 완료!
11
2385 의 11 번째 완료!
21
2385 의 21 번째 완료!
31
2385 의 31 번째 완료!
41
2385 의 41 번째 완료!
51
2385 의 51 번째 완료!
61
2385 의 61 번째 완료!
71
2385 의 71 번째 완료!
81
2385 의 81 번째 완료!
91
2385 의 91 번째 완료!
101
2385 의 101 번째 완료!
1
1023 의 1 번째 완료!
11
1023 의 11 번째 완료!
21
1023 의 21 번째 완료!
31
1023 의 31 번째 완료!
41
1023 의 41 번째 완료!
51
1023 의 51 번째 완료!
61
1023 의 61 번째 완료!
71
1023 의 71 번째 완료!
1
1032 의 1 번째 완료!
11
1032 의 11

AttributeError: 'NoneType' object has no attribute 'text'

In [16]:
crawling_article('삼성중공업')

1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31
1032 의 31 번째 완료!
41
1032 의 41 번째 완료!
51
1032 의 51 번째 완료!
1
1005 의 1 번째 완료!
11
1005 의 11 번째 완료!
21
1005 의 21 번째 완료!
31
1005 의 31 번째 완료!
41
1005 의 41 번째 완료!
1
1020 의 1 번째 완료!
11
1020 의 11 번째 완료!
21
1020 의 21 번째 완료!
31
1020 의 31 번째 완료!
41
1020 의 41 번째 완료!
1
1028 의 1 번째 완료!
11
1028 의 11 번째 완료!
21
1028 의 21 번째 완료!
31
1028 의 31 번째 완료!
1
2385 의 1 번째 완료!
11
2385 의 11 번째 완료!
21
2385 의 21 번째 완료!
31
2385 의 31 번째 완료!
41
2385 의 41 번째 완료!
51
2385 의 51 번째 완료!
61
2385 의 61 번째 완료!
71
2385 의 71 번째 완료!
81
2385 의 81 번째 완료!
91
2385 의 91 번째 완료!
1
1023 의 1 번째 완료!
11
1023 의 11 번째 완료!
21
1023 의 21 번째 완료!
31
1023 의 31 번째 완료!
41
1023 의 41 번째 완료!
1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31
1032 의 31 번째 완료!
41
1032 의 41 번째 완료!
1
1005 의 1 번째 완료!
11
1005 의 11 번째 완료!
21
1005 의 21 번째 완료!
31
1005 의 31 번째 완료!
41
1005 의 41 번째 완료!
51
1005 의 51 번째 완료!
1
1020 의 1 번째 완료!
11
1020 의 11 번째 완료!
21
1020 의 21 번째 완료!
31
1020 의 31 번째 완료!
41
1020 의 41 번째 완료

In [18]:
crawling_article('한화솔루션')

1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31
1032 의 31 번째 완료!
1
1005 의 1 번째 완료!
11
1005 의 11 번째 완료!
21
1005 의 21 번째 완료!
31
1005 의 31 번째 완료!
41
1005 의 41 번째 완료!
51
1005 의 51 번째 완료!
61
1005 의 61 번째 완료!
1
1020 의 1 번째 완료!
11
1020 의 11 번째 완료!
21
1020 의 21 번째 완료!
31
1020 의 31 번째 완료!
41
1020 의 41 번째 완료!
51
1020 의 51 번째 완료!
1
1028 의 1 번째 완료!
11
1028 의 11 번째 완료!
21
1028 의 21 번째 완료!
1
2385 의 1 번째 완료!
11
2385 의 11 번째 완료!
21
2385 의 21 번째 완료!
31
2385 의 31 번째 완료!
41
2385 의 41 번째 완료!
51
2385 의 51 번째 완료!
61
2385 의 61 번째 완료!
71
2385 의 71 번째 완료!
81
2385 의 81 번째 완료!
1
1023 의 1 번째 완료!
11
1023 의 11 번째 완료!
21
1023 의 21 번째 완료!
31
1023 의 31 번째 완료!
41
1023 의 41 번째 완료!
51
1023 의 51 번째 완료!
61
1023 의 61 번째 완료!
1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
1
1005 의 1 번째 완료!
11
1005 의 11 번째 완료!
21
1005 의 21 번째 완료!
31
1005 의 31 번째 완료!
41
1005 의 41 번째 완료!
1
1020 의 1 번째 완료!
11
1020 의 11 번째 완료!
21
1020 의 21 번째 완료!
31
1020 의 31 번째 완료!
41
1020 의 41 번째 완료!
51
1020 의 51 번째 완료!
61
1020 의 61 번째 완료

In [20]:
crawling_article('카카오')

1
1032 의 1 번째 완료!
11
1032 의 11 번째 완료!
21
1032 의 21 번째 완료!
31
1032 의 31 번째 완료!
41
1032 의 41 번째 완료!
51
1032 의 51 번째 완료!
61
1032 의 61 번째 완료!
71
1032 의 71 번째 완료!
81
1032 의 81 번째 완료!
91
1032 의 91 번째 완료!
101
1032 의 101 번째 완료!
111
1032 의 111 번째 완료!
121
1032 의 121 번째 완료!
131
1032 의 131 번째 완료!
141
1032 의 141 번째 완료!
151
1032 의 151 번째 완료!
161
1032 의 161 번째 완료!
171
1032 의 171 번째 완료!
181
1032 의 181 번째 완료!
191
1032 의 191 번째 완료!
201
1032 의 201 번째 완료!
211
1032 의 211 번째 완료!
221
1032 의 221 번째 완료!
231
1032 의 231 번째 완료!
241
1032 의 241 번째 완료!
251
1032 의 251 번째 완료!
261
1032 의 261 번째 완료!
271
1032 의 271 번째 완료!
281
1032 의 281 번째 완료!
291
1032 의 291 번째 완료!
301
1032 의 301 번째 완료!
311
1032 의 311 번째 완료!
321
1032 의 321 번째 완료!
331
1032 의 331 번째 완료!
341
1032 의 341 번째 완료!
351
1032 의 351 번째 완료!
361
1032 의 361 번째 완료!
371
1032 의 371 번째 완료!
381
1032 의 381 번째 완료!
391
1032 의 391 번째 완료!
401
1032 의 401 번째 완료!
411
1032 의 411 번째 완료!
421
1032 의 421 번째 완료!
431
1032 의 431 번째 완료!
441
1032 의 441 번째 완료!
451
1032 의 451 번째 완료!
461
1032 의

AttributeError: 'NoneType' object has no attribute 'text'

In [22]:
crawling_article('메리츠증권')

1
1032 의 1 번째 완료!
11


In [24]:
# 하병노
stock_list[:29] 

crawling_article('')
'''
['일진머티리얼즈', ㄴㄴ
 'LG화학', ㅇㅇ 에러
 '메리츠화재', ㅇㅇ 성공
 '삼성엔지니어링', ㅇㅇ
 '한국가스공사', ㅇㅇ
 '삼성중공업', ㅇㅇ 성공
 '한화솔루션', ㅇㅇ 성공
 '맥쿼리인프라', ㄴㄴ
 '카카오',
 'DB손해보험', ㅇㅇ성공
 'NAVER',
 '메리츠증권',
 '기아',
 '우리금융지주',
 '현대차',
 '이마트',
 '대한항공',
 '한솔케미칼',
 '아모레G',
 'LG디스플레이',
 '한국타이어앤테크놀로지',
 '현대건설',
 '쌍용C&E',
 '한온시스템',
 '삼성전자',
 '금호석유',
 'S-Oil',
 'POSCO',
 'GS']
 '''

['일진머티리얼즈',
 'LG화학',
 '메리츠화재',
 '삼성엔지니어링',
 '한국가스공사',
 '삼성중공업',
 '한화솔루션',
 '맥쿼리인프라',
 '카카오',
 'DB손해보험',
 'NAVER',
 '메리츠증권',
 '기아',
 '우리금융지주',
 '현대차',
 '이마트',
 '대한항공',
 '한솔케미칼',
 '아모레G',
 'LG디스플레이',
 '한국타이어앤테크놀로지',
 '현대건설',
 '쌍용C&E',
 '한온시스템',
 '삼성전자',
 '금호석유',
 'S-Oil',
 'POSCO',
 'GS']

In [25]:
# 김영은
stock_list[29:57]

'''
['LG전자',
 '한진칼',
 'LG생활건강',
 '고려아연',
 '한미사이언스',
 'LG',
 '현대제철',
 '롯데케미칼',
 '포스코케미칼',
 '현대중공업지주',
 'KT',
 'SK하이닉스',
 '신한지주',
 'LG이노텍',
 'KB금융',
 '넷마블',
 '삼성카드',
 'SK텔레콤',
 '한미약품',
 '엔씨소프트',
 '한국조선해양',
 'HMM',
 '코웨이',
 '한전기술',
 '롯데지주',
 '기업은행',
 'SK이노베이션',
 '하나금융지주']
'''

['LG전자',
 '한진칼',
 'LG생활건강',
 '고려아연',
 '한미사이언스',
 'LG',
 '현대제철',
 '롯데케미칼',
 '포스코케미칼',
 '현대중공업지주',
 'KT',
 'SK하이닉스',
 '신한지주',
 'LG이노텍',
 'KB금융',
 '넷마블',
 '삼성카드',
 'SK텔레콤',
 '한미약품',
 '엔씨소프트',
 '한국조선해양',
 'HMM',
 '코웨이',
 '한전기술',
 '롯데지주',
 '기업은행',
 'SK이노베이션',
 '하나금융지주']

In [26]:
# 김진섭
stock_list[57:]

'''
['삼성SDI',
 '삼성화재',
 '두산중공업',
 'CJ제일제당',
 '강원랜드',
 'GS건설',
 '오리온',
 '한국전력',
 '현대글로비스',
 'KT&G',
 'SKC',
 '삼성바이오로직스',
 'LG유플러스',
 '두산밥캣',
 '유한양행',
 '미래에셋증권',
 '한국금융지주',
 '삼성전기',
 '삼성물산',
 'SK',
 '아모레퍼시픽',
 'NH투자증권',
 '셀트리온',
 'GS리테일',
 '삼성생명',
 '삼성에스디에스',
 '메리츠금융지주',
 '현대모비스']
'''

['삼성SDI',
 '삼성화재',
 '두산중공업',
 'CJ제일제당',
 '강원랜드',
 'GS건설',
 '오리온',
 '한국전력',
 '현대글로비스',
 'KT&G',
 'SKC',
 '삼성바이오로직스',
 'LG유플러스',
 '두산밥캣',
 '유한양행',
 '미래에셋증권',
 '한국금융지주',
 '삼성전기',
 '삼성물산',
 'SK',
 '아모레퍼시픽',
 'NH투자증권',
 '셀트리온',
 'GS리테일',
 '삼성생명',
 '삼성에스디에스',
 '메리츠금융지주',
 '현대모비스']